In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Is this notebook running on Colab or Kaggle?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

if IS_COLAB or IS_KAGGLE:
    !apt update && apt install -y libpq-dev libsdl2-dev swig xorg-dev xvfb
    %pip install -U tf-agents pyvirtualdisplay
    %pip install -U gym>=0.21.0
    %pip install -U gym[box2d,atari,accept-rom-license]

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# To get smooth animations
import matplotlib.animation as animation
mpl.rc('animation', html='jshtml')


# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")


In [ ]:
# https://gym.openai.com/docs/
    
import gym

In [ ]:
# Criar o ambiente CartPole
# https://gym.openai.com/envs/CartPole-v1/

env = gym.make('CartPole-v1')

In [ ]:
# O ambiente é inicializado através da chamada ao método reset()
# O método devolve a primeira observação, no formato de um array NumPy com 4 valores reais
# (Posição horizontal, Velocidade, Ângulo de inclinação, Velocidade angular)


obs = env.reset(seed=42)

In [ ]:
# (Posição horizontal, Velocidade, Ângulo de inclinação, Velocidade Angular)

obs

In [ ]:
# Para garantir que o ambiente executa corretamente no Colab

try:
    import pyvirtualdisplay
    display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()
except ImportError:
    pass

In [ ]:
# Obter uma imagem do ambiente no formato NumPy Array
# Poderá também surgir um pop-up com a imagem. Se for esse o caso, podem ignorar essa janela


env.render()

In [ ]:
img = env.render(mode="rgb_array")
img.shape

In [ ]:
def plot_environment(env, figsize=(5,4)):
    plt.figure(figsize=figsize)
    img = env.render(mode="rgb_array")
    plt.imshow(img)
    plt.axis("off")
    return img

In [ ]:
# Visualizar a imagem obtida

plot_environment(env)
plt.show()

In [ ]:
# Verificar quantas ações é possível efetuar neste ambiente

env.action_space

In [ ]:
# 0 - accelerate left
# 1 - accelerate right

action = 1  

In [ ]:
# Executar uma ação
# Devolve nova observação, recompensa, informação sobre o final do episódio e outra informação 

obs, reward, done, info = env.step(action)
print('Obs: ', obs, '\nReward: ', reward, '\nDone: ', done, '\nInfo:', info)

In [ ]:
# Continuar a acelerar para a direita
# Verificar se as variações nas observações são consistentes com esta ação
# Eventualmente a simulação vai terminar

for i in range(50):
  obs, reward, done, info = env.step(action)
  print('Obs: ', obs, '\nReward: ', reward, '\nDone: ', done, '\nInfo:', info)
  if done == True:
    break


In [ ]:
# Quando termina a simulação, é necessário reinicializar o ambiente

if done:
    obs = env.reset(seed=42)

In [ ]:
###################################### PARTE 1 ######################################
# Definição de Políticas Fixas

# Política Reativa Simples
# Acelerar para tentar diminuir a inclinação do poste
# a observação obs[2] contém o ângulo de inclinação. 

def basic_policy(obs):
    angle = obs[2]
    return 0 if angle < 0 else 1

In [ ]:

# Simular 500 episódios 
# Recolher as recompensas em cada um destes episódios
# Cada episódio termina quando a variável done fica a true.

totals = []
for episode in range(500):
    episode_rewards = 0
    obs = env.reset()
    for step in range(200):
        action = basic_policy(obs)
        obs, reward, done, info = env.step(action)
        episode_rewards += reward
        if done:
            break
    totals.append(episode_rewards)

In [ ]:
# Verificar o desempenho médio e a duração dos episódios com maior e menor duração
# Analise os resultados

np.mean(totals), np.std(totals), np.min(totals), np.max(totals)

In [ ]:
# Executar um episódio e recolher imagens em cada um dos passos

frames = []

obs = env.reset(seed=42)
for step in range(200):
    img = env.render(mode="rgb_array")
    frames.append(img)
    action = basic_policy(obs)

    obs, reward, done, info = env.step(action)
    if done:
        break

In [ ]:
def update_scene(num, frames, patch):
    patch.set_data(frames[num])
    return patch,

def plot_animation(frames, repeat=False, interval=40):
    fig = plt.figure()
    patch = plt.imshow(frames[0])
    plt.axis('off')
    anim = animation.FuncAnimation(
        fig, update_scene, fargs=(frames, patch),
        frames=len(frames), repeat=repeat, interval=interval)
    plt.close()
    return anim

In [ ]:
# Visualizar um episódio

plot_animation(frames)

In [ ]:
# Politica Reativa Melhorada

# Complete o método not_so_basic_policy(obs) para tentar definir uma política fixa com melhor desempenho do que a anterior

import random

# Para já o método está a devolver cada uma das ações com probabilidade 0.5, ignorando a observação
# Altere o que achar conveniente para melhorar o desempenho

def not_so_basic_policy(obs):
  return random.randint(0, 1)

In [ ]:
# Simular 500 episódios com a nova politica
# Compare os resultados obtidos com os da politica básica

totals = []

for episode in range(500):
    episode_rewards = 0
    obs = env.reset()
    for step in range(200):
        action = not_so_basic_policy(obs)
        obs, reward, done, info = env.step(action)
        episode_rewards += reward
        if done:
            break
    totals.append(episode_rewards)

np.mean(totals), np.std(totals), np.min(totals), np.max(totals)

In [ ]:
###################################### PARTE 2 ######################################

# Neural Network Policies

from tensorflow import keras
from tensorflow.keras import layers

keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)


In [ ]:
# Criar uma rede neuronal básica que recebe uma observação (4 valores) e devolve a probabilidade de executar a ação 0

n_inputs = 4 

model = keras.models.Sequential([
    keras.layers.Dense(5, activation="elu", input_shape=[n_inputs]),
    keras.layers.Dense(1, activation="sigmoid"),
])

In [ ]:
 # Função que executa um episódio e recolhe as imagens para visualização numa animação

def render_policy_net(model, n_max_steps=200, seed=42):
    frames = []
    env = gym.make("CartPole-v1")
    np.random.seed(seed)
    obs = env.reset(seed=seed)
    for step in range(n_max_steps):
        frames.append(env.render(mode="rgb_array"))
        left_proba = model.predict(obs.reshape(1, -1))
        action = int(np.random.rand() > left_proba)
        obs, reward, done, info = env.step(action)
        if done:
            print('Performed', step, 'steps')
            break
    env.close()
    return frames

In [ ]:
# Avaliar o desempenho da rede
# Ela ainda não foi treinada e terá um desempenho aleatório

frames = render_policy_net(model)
plot_animation(frames)

In [ ]:
# Aprender uma estratégia básica -> Problema resolvido como uma situação supervisionada
# Pretende-se que a rede aprenda uma estratégia igual à estratégia básica que implementámos anteriormente
# Neste caso o target será definido em função do ângulo de inclinação do poste (obs[2]).

# Treinar em 50 ambientes diferentes, durante 5000 iterações

n_environments = 50
n_iterations = 5000

envs = [gym.make("CartPole-v1") for _ in range(n_environments)]
for index, env in enumerate(envs):
    env.seed(index)
np.random.seed(42)
observations = [env.reset() for env in envs]
optimizer = keras.optimizers.RMSprop()
loss_fn = keras.losses.binary_crossentropy

for iteration in range(n_iterations):
    # Define the supervised target -> if angle < 0, we want proba(left) = 1., or else proba(left) = 0.  
    target_probas = np.array([([1.] if obs[2] < 0 else [0.])
                              for obs in observations])
    with tf.GradientTape() as tape:
        left_probas = model(np.array(observations))
        loss = tf.reduce_mean(loss_fn(target_probas, left_probas))
    print("\rIteration: {}, Loss: {:.3f}".format(iteration, loss.numpy()), end="")
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    actions = (np.random.rand(n_environments, 1) > left_probas.numpy()).astype(np.int32)
    for env_index, env in enumerate(envs):
        obs, reward, done, info = env.step(actions[env_index][0])
        observations[env_index] = obs if not done else env.reset()

for env in envs:
    env.close()

In [ ]:
# Avaliar o desempenho do modelo treinado
# Mudar a seed s para testar o comportamento em ambientes diferentes

s = 10

frames = render_policy_net(model, 200, s)
plot_animation(frames)

In [ ]:
# Implementar uma política baseada em gradientes, utilizando uma variante do método REINFORCE

# Método auxiliar para executar um passo de um episódio
# Guarda os gradientes e a loss, mas não os aplica

def play_one_step(env, obs, model, loss_fn):
    with tf.GradientTape() as tape:
        left_proba = model(obs[np.newaxis])
        action = (tf.random.uniform([1, 1]) > left_proba)
        y_target = tf.constant([[1.]]) - tf.cast(action, tf.float32)
        loss = tf.reduce_mean(loss_fn(y_target, left_proba))
    grads = tape.gradient(loss, model.trainable_variables)
    obs, reward, done, info = env.step(int(action[0, 0].numpy()))
    return obs, reward, done, grads


# Método que permite executar vários episódios e guardar os gradientes e as recompensas

def play_multiple_episodes(env, n_episodes, n_max_steps, model, loss_fn):
    all_rewards = []
    all_grads = []
    for episode in range(n_episodes):
        current_rewards = []
        current_grads = []
        obs = env.reset(seed=42)
        for step in range(n_max_steps):
            obs, reward, done, grads = play_one_step(env, obs, model, loss_fn)
            current_rewards.append(reward)
            current_grads.append(grads)
            if done:
                break
        all_rewards.append(current_rewards)
        all_grads.append(current_grads)
    return all_rewards, all_grads

# Métodos auxiliares para processamento de recompensas

def discount_rewards(rewards, discount_rate):
    discounted = np.array(rewards)
    for step in range(len(rewards) - 2, -1, -1):
        discounted[step] += discounted[step + 1] * discount_rate
    return discounted

def discount_and_normalize_rewards(all_rewards, discount_rate):
    all_discounted_rewards = [discount_rewards(rewards, discount_rate)
                              for rewards in all_rewards]
    flat_rewards = np.concatenate(all_discounted_rewards)
    reward_mean = flat_rewards.mean()
    reward_std = flat_rewards.std()
    return [(discounted_rewards - reward_mean) / reward_std
            for discounted_rewards in all_discounted_rewards]


In [ ]:
# Criar e preparar a rede para o treino com o algoritmo indicado

n_iterations = 150
n_episodes_per_update = 10
n_max_steps = 200
discount_rate = 0.95

optimizer = keras.optimizers.Adam(learning_rate=0.01)
loss_fn = keras.losses.binary_crossentropy

keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Dense(5, activation="elu", input_shape=[4]),
    keras.layers.Dense(1, activation="sigmoid"),
])

In [ ]:
# Treinar
# Verificar como a recompensa média vai aumentando ao longo das iterações de treino

env = gym.make("CartPole-v1")

for iteration in range(n_iterations):
    all_rewards, all_grads = play_multiple_episodes(
        env, n_episodes_per_update, n_max_steps, model, loss_fn)
    total_rewards = sum(map(sum, all_rewards))                     
    print("\rIteration: {}, mean rewards: {:.1f}".format(          
        iteration, total_rewards / n_episodes_per_update), end="") 
    all_final_rewards = discount_and_normalize_rewards(all_rewards,
                                                       discount_rate)
    all_mean_grads = []
    for var_index in range(len(model.trainable_variables)):
        mean_grads = tf.reduce_mean(
            [final_reward * all_grads[episode_index][step][var_index]
             for episode_index, final_rewards in enumerate(all_final_rewards)
                 for step, final_reward in enumerate(final_rewards)], axis=0)
        all_mean_grads.append(mean_grads)
    optimizer.apply_gradients(zip(all_mean_grads, model.trainable_variables))

env.close()

In [ ]:
# Avaliar o desempenho do modelo treinado com o algoritmo REINFORCE
# Mudar a seed s para testar o comportamento em ambientes diferentes

s = 41

frames = render_policy_net(model, 200, s)
plot_animation(frames)